In [ ]:
!pip install mimikit

In [ ]:
from mimikit.freqnet import FreqNet
from mimikit.data import Database, download_database
from mimikit.utils import audio
from mimikit import get_trainer, get_token

In [ ]:
# neptune :

neptune_token = get_token()

# for the db :

neptune_project_db = "k-tonal/data-and-base-notebooks"
neptune_exp_id_db = "DAT-27"
full_exp_path_db = neptune_project_db + "/" + neptune_exp_id_db
db_name = "genoel-mix.h5"

# for the model :

neptune_project = "k-tonal/model-upload-test"

# paths

path_to_db = "../" + db_name
path_to_model = "../experiment-1"


In [ ]:
# get your data

# db = download_database(neptune_token,
#                        full_exp_path_db,
#                        database_name=db_name,
#                        destination="./")
db = Database(path_to_db)
db.metadata

### Varying Parameters

In [ ]:
params = {
    # from quick to slow without pad and without accum :
    0: dict(
        with_skip_conv=False,
        with_residual_conv=False,
        pad_input=0,
        accum_outputs=0,
    ),
    1: dict(
        with_skip_conv=False,
        with_residual_conv=True,
        pad_input=0,
        accum_outputs=0,
    ),
    2:  dict(
        with_skip_conv=True,
        with_residual_conv=False,
        pad_input=0,
        accum_outputs=0,
    ),
    3:  dict(
        with_skip_conv=True,
        with_residual_conv=True,
        pad_input=0,
        accum_outputs=0,
    ),
    # from quick to slow with accum on the right
    4: dict(
        with_skip_conv=False,
        with_residual_conv=False,
        pad_input=0,
        accum_outputs=-1,
    ),
    5: dict(
        with_skip_conv=False,
        with_residual_conv=True,
        pad_input=0,
        accum_outputs=-1,
    ),
    6:  dict(
        with_skip_conv=True,
        with_residual_conv=False,
        pad_input=0,
        accum_outputs=-1,
    ),
    7:  dict(
        with_skip_conv=True,
        with_residual_conv=True,
        pad_input=0,
        accum_outputs=-1,
    ),
    # only quick with pad left or right & with or without accum
    8: dict(
        with_skip_conv=False,
        with_residual_conv=False,
        pad_input=-1,
        accum_outputs=1,
    ),
    9: dict(
        with_skip_conv=False,
        with_residual_conv=False,
        pad_input=1,
        accum_outputs=1,
    ),
    10:  dict(
        with_skip_conv=False,
        with_residual_conv=False,
        pad_input=1,
        accum_outputs=0,
    ),
    11:  dict(
        with_skip_conv=False,
        with_residual_conv=False,
        pad_input=-1,
        accum_outputs=0,
    ),
    # only slow with pad left or right & with or without accum
    12: dict(
        with_skip_conv=True,
        with_residual_conv=True,
        pad_input=-1,
        accum_outputs=1,
    ),
    13: dict(
        with_skip_conv=True,
        with_residual_conv=True,
        pad_input=1,
        accum_outputs=1,
    ),
    14:  dict(
        with_skip_conv=True,
        with_residual_conv=True,
        pad_input=1,
        accum_outputs=0,
    ),
    15:  dict(
        with_skip_conv=True,
        with_residual_conv=True,
        pad_input=-1,
        accum_outputs=0,
    ),
}

In [ ]:
# pick a parametrization :

PARAMS_INDEX = 9

# build a model and its trainer

model = FreqNet(
    data_object=db.fft,
    splits=[.85, .15],
     model_dim=1024,
     groups=1,
     n_layers=(4,),
    **params[PARAMS_INDEX]
     )

trainer = get_trainer(model=model,
                     max_epochs=100,
                     epochs=[100],
                     # try everything in 16-bits precision
                     precision=16,
                     root_dir=path_to_model,
                     neptune_api_token=neptune_token,
                     neptune_project=neptune_project,
                     )

# log some infos to the neptune experiment :

nept_exp = [exp for exp in trainer.logger.experiment if exp.__class__.__name__ == "Experiment"]
if any(nept_exp):
    nept_exp[0].append_tags("INDEX=" + str(PARAMS_INDEX),
                         "precision=16",
                         "FreqNet",
                         "db="+db_name,
                         "db_size=" + str(model.data.shape[0]),
                         *[k + "=" + str(v) for k, v in params[PARAMS_INDEX].items()])
    nept_exp[0].log_text("DB", db_name)
    nept_exp[0].log_text("INDEX", str(PARAMS_INDEX))
    

# Train :

trainer.fit(model)

# upload checkpoints and logs

model.upload_to_neptune(artifacts=("states", "logs"))

### try some prompts

In [ ]:
from random import randint

n_prompts = 3
prompts_indices = [randint(0, model.data.shape[0]-64) for _ in range(n_prompts)]

for n, i in enumerate(prompts_indices):
    output = model.generate(model.data[i:i+64],
                            n_steps=2048)
    
    print("prompt", n, "index", i)
    
    audio(output.squeeze().numpy())


### log a prompt you like

In [ ]:
i = 1234

model.log_audio("prompt=" + str(i), model.generate(model.data[i:i+64], n_steps=2048))